In [13]:
import jedi
import re

from llmcoder.analyze.MypyAnalyzer import MypyAnalyzer

In [14]:
source = """
import tiktoken
import numpy as np

example_dict = {
    "enc": tiktoken.Tiktoken()
}

rand_int = np.random.randiint(0, 10, 1)
"""

script = jedi.Script(source)

In [15]:
# script.complete_search("np.") # https://www.phind.com/search?cache=ey5i26k2mr5wuezcjx9tzkaf

In [16]:
mypy_analyzer = MypyAnalyzer()

In [17]:
analysis_results = mypy_analyzer.analyze(input = "", completion = source)

In [18]:
def extract_modules(s):
    pattern = r'([a-zA-Z0-9_]*)\.([a-zA-Z0-9_]*)'
    match = re.search(pattern, s)
    if match:
        return match.group(1) + '.' + match.group(2)
    else:
        return None

In [21]:
for line in analysis_results["message"].split("\n"):
    if line.startswith("your completion:"):
        error_line_number = int(line.split(":")[1])
        error_line = source.split("\n")[error_line_number - 1]

        matches_no_attr = re.findall(r'has no attribute \"(.+?)\"', line)

        for match in matches_no_attr:
            if match.startswith("type["):
                match = match[5:-1]

        print(error_line, matches_no_attr)

        # Get the module/class name of the hallucinated attribute from the error_line and the match
        # E.g. "tiktoken" in "tiktoken.Tiktoken" or numpy.random" in "numpy.random.randint"
        module_matches = re.findall(r'([a-zA-Z0-9_]*)\.([a-zA-Z0-9_]*)', error_line)

        if len(module_matches) == 0:
            continue

        hallucinated_attribute_module = '.'.join(module_matches[0][:-1])

        print(hallucinated_attribute_module)

        try:
            suggested_attributes = script.complete_search(hallucinated_attribute_module + '.')
        except AttributeError:
            continue

        results_str = ""
        for suggested_attribute in suggested_attributes:
            results_str += suggested_attribute.name + ", "

            

    "enc": tiktoken.Tiktoken() ['Tiktoken']
tiktoken
[<Completion: Encoding>, <Completion: encoding_for_model>, <Completion: encoding_name_for_model>, <Completion: get_encoding>, <Completion: list_encoding_names>, <Completion: registry>, <Completion: core>, <Completion: load>, <Completion: model>, <Completion: _educational>, <Completion: tiktoken._tiktoken>, <Completion: __package__>, <Completion: __doc__>, <Completion: __name__>, <Completion: __file__>]
---
rand_int = np.random.randiint(0, 10, 1) ['randiint']
np


In [25]:
suggested_attributes[0].name

'Encoding'